In [2]:
%matplotlib inline
from __future__ import division
import pandas as pd
import seaborn as sns
import networkx as nx
import numpy as np
import itertools
import os
import matplotlib.pyplot as plt
from ripe_interface import get_db
from process_big_traceroutes import big_process, non_responsive_generator
from mappings import get_country, get_asn
from ripe.atlas.sagan.traceroute import TracerouteResult

## Get the appropiate databse

In [3]:
db = get_db('rala-40')

### Set the default directory: "data"

In [4]:
default_dir = '../../home/app/data/'

### Set the file where traces are going to be saved

In [5]:
WRITE_TRACE_FILE = default_dir + 'sample.traces'

### Read from the database, each result and write it, csv in WRITE_TRACE_FILE

In [9]:
non_gen = non_responsive_generator()
with open(WRITE_TRACE_FILE, "w") as f:                                    
    count = 0                                                           
    for r in db.results.find({}): 
        count = count + 1
        if count == 10:  # Comment this to get all of results
            break  # Also this
        tr = TracerouteResult(r)                               
        y = big_process(tr,non_gen,add_first=False, keep_last=False, keep_none=False)
        f.write("H,{},{}\n".format(tr.origin, tr.destination_address))  
        for i,x in enumerate(y):                                      
            if x is not None and x.ip is not None:
                f.write("B,{},{},{},{}\n".format(i,str(x.ip),x.pack.rtt, x.pack.ttl))                                                           

### Read the written file and for each couple of consecutive lines infere a link (useful if not keeping first,last)

In [7]:
links = []  
with open(WRITE_TRACE_FILE, "r") as f:  
    ini, fin = None, None
    line1, line2 = itertools.tee(f)
    next(line2)
    for x,y in zip(line1,line2):
        l1, l2 = x.strip(), y.strip()
        if l1[0] == "B" and l2[0] == "B":                 
            _, pos1, ip1, rtt1, ttl1 = l1.split(",")       
            _, pos2, ip2, rtt2, ttl2 = l2.split(",")              
            links.append((ip1,ip2))

In [10]:
links = []  
with open(WRITE_TRACE_FILE, "r") as f:  
    ini, fin = None, None
    for l in f:
        line1 = l.strip()
        if line1[0] == "H":
            if ini is not None:
                links.append((ini,fin))

            _, ini, fin = line1.split(",")
        else:             
            _, pos1, ip1, rtt1, ttl1 = line1.split(",")         
            links.append((ini,ip1))
            ini = ip1

In [11]:
links[:20]

[('2800:a4:1400:6b00:a2f3:c1ff:fec4:4900',
  '2800:a4:1400:6b01:8ee1:17ff:fefe:1ec6'),
 ('2800:a4:1400:6b01:8ee1:17ff:fefe:1ec6', '2800:a0:301:c828:a10c:1f00:0:c'),
 ('2800:a0:301:c828:a10c:1f00:0:c', '2001:12f8::220:19'),
 ('2001:12f8::220:19', '2001:12f8::217:48'),
 ('2001:12f8::217:48', '2001:1291:0:53::b'),
 ('2001:1291:0:53::b', '2001:1291:0:15b::a'),
 ('2001:1291:0:15b::a', '2001:1291:0:100::b'),
 ('2001:1291:0:100::b', '2001:1291:0:74::b'),
 ('2001:1291:0:74::b', '2001:1291:1601:37::b'),
 ('2001:1318:400:0:6666:b3ff:feb0:d110', '2001:1318:400::1'),
 ('2001:1318:400::1', '2001:1318::9'),
 ('2001:1318::9', '2001:1318::5'),
 ('2001:1318::5', '2800:c30:b5d1::81'),
 ('2800:c30:b5d1::81', '2001:450:2002:802::2'),
 ('2001:450:2002:802::2', '2001:450:2002:802::1'),
 ('2001:450:2002:802::1', '2001:450:2001:1000:0:670:1708:3080'),
 ('2001:450:2001:1000:0:670:1708:3080', '2804:4c0:ffff:fc7f::2'),
 ('2804:4c0:ffff:fc7f::2', '2001:1291:0:150::b'),
 ('2001:1291:0:150::b', '2001:1291:0:138::b'

### Write links to file

In [16]:
with open(default_dir+'sample.links', 'w') as f:
    for (v1,v2) in links:
        f.write('{},{}\n'.format(v1,v2))

## Create graph from links

In [10]:
G = nx.Graph()
G.add_edges_from(links)

#### Compute some basic properties 

In [14]:
print 'Number of nodes:', G.number_of_nodes()
print 'Number of edges:', G.number_of_edges()

Number of nodes: 63
Number of edges: 56


### Alternativly load them from the saved file

In [17]:
with open(default_dir+'sample.links', 'r') as f: G2 = nx.read_edgelist(f, delimiter=',')

In [19]:
nx.is_isomorphic(G,G2)  # They are the same graph

True